<a href="https://colab.research.google.com/github/gabrielaugustavo/LLM/blob/main/GSI073_aula0_regressao_logistica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSI073 - Tópicos Especiais de Inteligência Artificial

## Definição dos dados

In [ ]:
import torch; import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X = iris.data        # 4 features: sépalas e pétalas
y = (iris.target == 1).astype(float)  # 1 se Versicolor, 0 caso contrário

print(X)
# 2. Preparar dados para pytorch
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, random_state=44, stratify=y
)

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float().view(-1, 1)

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float().view(-1, 1)


## Definição do modelo e treinamento

In [ ]:
# 3. Definir modelo: regressão logística
modelo = torch.nn.Linear(4, 1)  # 4 features → 1 saída (probabilidade de ser Versicolor)

# 4. Definir função de perda e algoritmo de otimização
funcao_perda = torch.nn.BCEWithLogitsLoss()  # combinação de sigmoid + BCE
optimizer = torch.optim.SGD(modelo.parameters(), lr=0.1)

## Execução do treinamento

In [ ]:
# 5. Treino
for epoch in range(5000):
    optimizer.zero_grad() # reseta gradiente senão acumula
    outputs = modelo(X_train)
    loss = funcao_perda(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Época [{epoch+1}/100], Loss: {loss.item():.4f}")

In [ ]:
with torch.no_grad():
    logits = modelo(X_test)
    probs = torch.sigmoid(logits)

    preds = (probs >= 0.5).float()  # converte probabilidade em 0 ou 1
    print(preds)
    acuracia = (preds.eq(y_test).sum() / y_test.shape[0]).item()

print("\n=== Avaliação do Modelo ===")
print(f"Acurácia no conjunto de teste: {acuracia*100:.2f}%")

entrada = torch.tensor([[8, 2.7, 4.1, 1.0]], dtype=torch.float32)
probabilidade = torch.sigmoid(modelo(entrada)).item()
print(f"Probabilidade de ser Versicolor: {probabilidade*100:.2f}%")




In [ ]:

with torch.no_grad():
    logits = modelo(X_test)
    probs = torch.sigmoid(logits).numpy().flatten()
    y_true = y_test.numpy().flatten()


thresholds = np.linspace(0, 1, 101)
melhor_thresh = 0.5
melhor_acc = 0

for t in thresholds:
    preds = (probs >= t).astype(float)
    acc = (preds == y_true).mean()  # média dos acertos

    if acc > melhor_acc:
        melhor_acc = acc
        melhor_thresh = t

print(f"Melhor threshold: {melhor_thresh:.2f}")
print(f"Acurácia máxima: {melhor_acc*100:.2f}%")

In [ ]:


fpr, tpr, thresholds = roc_curve(y_true, probs)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, label=f'ROC (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate (Recall)')
plt.title('Curva ROC')
plt.legend()
plt.grid(True)
plt.show()
